In [36]:
import pandas as pd; import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings; import os 

import tensorflow as tf 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import optimizers
building_id = 1

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

t_model_state_keys = ['Temp_mean_p0','Temp_mean_m1','Occ','temp_OA','rh_OA','hir_sol','u_heat_stp','u_cool_stp']
t_model_pred_sel =  ['delta_T','hvac_dem_tot']

t_keys = list(set(t_model_state_keys+t_model_pred_sel))

min_max = {}

if building_id == 1: 
    data = '0_Data/Case_1/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,6020] #1  #kW 
    min_max['appliances_pow'] = [0,42] #1 #kW
    min_max['elec_lights'] = [0,70]   #1 #kW
    min_max['Occ'] = [0,3300]
    scale_delta, scale_hvac = 200 , 200
elif building_id == 6:
    data = '0_Data/Case_6/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,1620] #6  #kW 
    min_max['appliances_pow'] = [0,160] #6 #kW
    min_max['elec_lights'] = [0,190]   #6 #kW
    min_max['Occ'] = [0,250]
    scale_delta, scale_hvac = 600 , 200
      

min_max['u_heat_stp'] = [15,25] 
min_max['u_cool_stp'] = [20,30] 
min_max['delta_T'] = [0,1] 
min_max['Stp_n'] = [0,1] 
min_max['mode'] = [0,1] 

# lighting and appliances follow a random scale where the pred values are limited between 0-10
min_max['elec_equip'] = [0,35]    #kW

# lighting and appliances follow a random scale where the pred values are limited between 0-10


def training_override(row):
    T = row['Temp_mean_p0']
    htg = row['u_heat_stp']
    clg = row['u_cool_stp']
    T_oa = row['temp_OA']
    if T<htg and htg < clg:
        clg = min_max['u_cool_stp'][1]
    elif T>htg and htg<clg:
        if T_oa> min_max['u_cool_stp'][0]:
            htg = min_max['u_heat_stp'][0]
        elif T_oa< min_max['u_heat_stp'][0]:
            clg = min_max['u_cool_stp'][1]
    elif T>clg and htg<clg:
        htg = min_max['u_heat_stp'][0]
    return [htg,clg]



# def training_override(T,htg,clg,T_oa):
#     if T<htg and htg < clg:
#         # print ("check 1")
#         clg = min_max['u_cool_stp'][1]
#         # print ("htg: {}, clg: {}".format(htg,clg))
#     elif T>htg and htg<clg:
#         # print ("check 2")
#         if T_oa> min_max['u_cool_stp'][0]:
#             htg = min_max['u_heat_stp'][0]
#             # print ("check 3")
#         elif T_oa< min_max['u_heat_stp'][0]:
#             clg = min_max['u_cool_stp'][1]
#             # print ("check 4")
#     elif T>clg and htg<clg:
#         htg = min_max['u_heat_stp'][0]
#         # print ("check 5")
#     return htg,clg 


In [38]:
path_save = '0_Data/Case_1/'
df = pd.read_csv(path_save+"df.csv")

In [39]:
df.columns 

Index(['Unnamed: 0', 'index', 'Date/Time', 'temp_OA', 'rh_OA', 'hir_sol',
       'Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](TimeStep)',
       'Environment:Site Direct Solar Radiation Rate per Area [W/m2](TimeStep)',
       'Environment:Site Total Sky Cover [](TimeStep)',
       'Environment:Site Opaque Sky Cover [](TimeStep)',
       '0_UNCOND_ZONE:Zone Total Internal Total Heating Rate [W](TimeStep)',
       '1_BOTTOM_ZONE:Zone Total Internal Total Heating Rate [W](TimeStep)',
       '2_MID_ZONE:Zone Total Internal Total Heating Rate [W](TimeStep)',
       '3_UPPER_PENTHOUSE:Zone Total Internal Total Heating Rate [W](TimeStep)',
       'Occ_Z0', 'Occ_Z1', 'Occ_Z2', 'Occ_Z3',
       '0_UNCOND_ZONE:Zone Infiltration Total Heat Loss Energy [J](TimeStep)',
       '0_UNCOND_ZONE:Zone Infiltration Total Heat Gain Energy [J](TimeStep)',
       '0_UNCOND_ZONE:Zone Infiltration Mass [kg](TimeStep)',
       '1_BOTTOM_ZONE:Zone Infiltration Total Heat Loss Energy [J](TimeSte

In [35]:
df['delta_T'].idxmax()

106149

In [40]:
view_no = 20
start = df['delta_T'].idxmax()

df.iloc[start:start+view_no][['Temp_mean_p0','Temp_mean_m1','Occ','temp_OA','rh_OA','hir_sol','u_heat_stp','u_cool_stp','delta_T','hvac_dem_tot']]

,Temp_mean_p0,Temp_mean_m1,Occ,temp_OA,rh_OA,hir_sol,u_heat_stp,u_cool_stp,delta_T,hvac_dem_tot
106149,22.565957,17.406807,0.0,-5.6,93.333333,257.333333,25.0,28.0,5.159150,6441.931008
106150,25.000754,22.565957,0.0,-5.6,94.666667,257.666667,25.0,28.0,2.434797,2819.428396
106151,25.048121,25.000754,0.0,-5.6,96.000000,258.000000,25.0,28.0,0.047367,3123.277373
106152,25.021823,25.048121,0.0,-5.6,97.333333,258.333333,25.0,28.0,-0.026298,2827.731547
106153,25.004906,25.021823,0.0,-5.6,98.666667,258.666667,25.0,28.0,-0.016917,2495.502825
106154,25.001086,25.004906,0.0,-5.6,100.000000,259.000000,25.0,28.0,-0.003820,2462.107724
106155,23.829428,25.001086,0.0,-5.5,99.333333,259.333333,23.0,27.0,-1.171658,1215.457473
106156,23.434374,23.829428,0.0,-5.4,98.666667,259.666667,23.0,27.0,-0.395053,1392.110407
106157,23.079314,23.434374,0.0,-5.3,98.000000,260.000000,23.0,27.0,-0.355060,1610.886076
106158,23.001398,23.079314,0.0,-5.2,97.333333,260.333333,23.0,27.0,-0.077916,2005.976397


In [41]:
df['stp'] = df.apply(training_override,axis=1)
df['u_heat_stp'] = df.apply(lambda x: x['stp'][0],axis=1)
df['u_cool_stp'] = df.apply(lambda x: x['stp'][1],axis=1)

df.iloc[start-1:start+view_no][['Temp_mean_p0','Temp_mean_m1','Occ','temp_OA','rh_OA','hir_sol','u_heat_stp','u_cool_stp','delta_T','hvac_dem_tot','stp']]


,Temp_mean_p0,Temp_mean_m1,Occ,temp_OA,rh_OA,hir_sol,u_heat_stp,u_cool_stp,delta_T,hvac_dem_tot,stp
106148,17.406807,17.486607,0.0,-5.6,92.000000,257.000000,17.0,30.0,-0.079801,637.244709,"[17.0, 30]"
106149,22.565957,17.406807,0.0,-5.6,93.333333,257.333333,25.0,30.0,5.159150,6441.931008,"[25.0, 30]"
106150,25.000754,22.565957,0.0,-5.6,94.666667,257.666667,25.0,30.0,2.434797,2819.428396,"[25.0, 30]"
106151,25.048121,25.000754,0.0,-5.6,96.000000,258.000000,25.0,30.0,0.047367,3123.277373,"[25.0, 30]"
106152,25.021823,25.048121,0.0,-5.6,97.333333,258.333333,25.0,30.0,-0.026298,2827.731547,"[25.0, 30]"
106153,25.004906,25.021823,0.0,-5.6,98.666667,258.666667,25.0,30.0,-0.016917,2495.502825,"[25.0, 30]"
106154,25.001086,25.004906,0.0,-5.6,100.000000,259.000000,25.0,30.0,-0.003820,2462.107724,"[25.0, 30]"
106155,23.829428,25.001086,0.0,-5.5,99.333333,259.333333,23.0,30.0,-1.171658,1215.457473,"[23.000000000000004, 30]"
106156,23.434374,23.829428,0.0,-5.4,98.666667,259.666667,23.0,30.0,-0.395053,1392.110407,"[22.99999999999999, 30]"
106157,23.079314,23.434374,0.0,-5.3,98.000000,260.000000,23.0,30.0,-0.355060,1610.886076,"[22.99999999999999, 30]"


In [6]:
df.head()

,Unnamed: 0,index,Date/Time,temp_OA,wb_temp_OA,rh_OA,wind_speed,wind_dir,hir_sol,dif_sol,...,date_obj,sen_hou,day_of_year,day_no,day_secs,Temp_mean_p0,Temp_mean_m1,delta_T,tot_secs,stp
0,0,1009,01/01 00:20:00,7.0,6.001802,87.0,4.433333,276.666667,323.0,0.0,...,2006-01-01 00:20:00,0.333333,1,1,1200,21.134384,21.487900,0.287331,1200,"[21.0, 30]"
1,1,1010,01/01 00:30:00,8.3,7.005396,84.0,4.350000,255.000000,330.0,0.0,...,2006-01-01 00:30:00,0.500000,1,1,1800,21.421715,21.134384,2.977642,1800,"[25.000000000000007, 30]"
2,2,1011,01/01 00:40:00,9.6,7.982120,81.0,4.266667,233.333333,337.0,0.0,...,2006-01-01 00:40:00,0.666667,1,1,2400,24.399357,21.421715,-1.083755,2400,"[23.000000000000018, 30]"
3,3,1012,01/01 00:50:00,10.9,8.930928,78.0,4.183333,211.666667,344.0,0.0,...,2006-01-01 00:50:00,0.833333,1,1,3000,23.315601,24.399357,1.583622,3000,"[25.0, 30]"
4,4,1013,01/01 01:00:00,12.2,9.850736,75.0,4.100000,190.000000,351.0,0.0,...,2006-01-01 01:00:00,1.000000,1,1,3600,24.899224,23.315601,-1.640921,3600,"[21.0, 30]"


In [9]:
df['hvac_dem_tot'].max()

1614.6254671909344

In [43]:
df_n = df[:]

for key in t_keys:
    df_n[key] = (df_n[key] - min_max[key][0])/(min_max[key][1]-min_max[key][0])

In [44]:
df_n.iloc[start:start+view_no][['Temp_mean_p0','Temp_mean_m1','Occ','temp_OA','rh_OA','hir_sol','u_heat_stp','u_cool_stp','delta_T','hvac_dem_tot']]

,Temp_mean_p0,Temp_mean_m1,Occ,temp_OA,rh_OA,hir_sol,u_heat_stp,u_cool_stp,delta_T,hvac_dem_tot
106149,0.504397,0.160454,0.0,0.240000,0.933333,0.367619,1.0,1.0,5.159150,1.070088
106150,0.666717,0.504397,0.0,0.240000,0.946667,0.368095,1.0,1.0,2.434797,0.468344
106151,0.669875,0.666717,0.0,0.240000,0.960000,0.368571,1.0,1.0,0.047367,0.518817
106152,0.668122,0.669875,0.0,0.240000,0.973333,0.369048,1.0,1.0,-0.026298,0.469723
106153,0.666994,0.668122,0.0,0.240000,0.986667,0.369524,1.0,1.0,-0.016917,0.414535
106154,0.666739,0.666994,0.0,0.240000,1.000000,0.370000,1.0,1.0,-0.003820,0.408988
106155,0.588629,0.666739,0.0,0.241667,0.993333,0.370476,0.8,1.0,-1.171658,0.201903
106156,0.562292,0.588629,0.0,0.243333,0.986667,0.370952,0.8,1.0,-0.395053,0.231248
106157,0.538621,0.562292,0.0,0.245000,0.980000,0.371429,0.8,1.0,-0.355060,0.267589
106158,0.533427,0.538621,0.0,0.246667,0.973333,0.371905,0.8,1.0,-0.077916,0.333219


In [45]:
scale_delta, scale_hvac = 200 , 200
df_n['delta_T'] = df_n['delta_T'] * scale_delta
df_n['hvac_dem_tot'] = df_n['hvac_dem_tot'] * scale_hvac

therm_X = df_n[t_model_state_keys]
therm_y = df_n[t_model_pred_sel]

In [46]:
therm_X = df_n[t_model_state_keys]
therm_y = df_n[t_model_pred_sel]

pd.concat([therm_X,therm_y],axis=1).iloc[start:start+view_no]

,Temp_mean_p0,Temp_mean_m1,Occ,temp_OA,rh_OA,hir_sol,u_heat_stp,u_cool_stp,delta_T,hvac_dem_tot
106149,0.504397,0.160454,0.0,0.240000,0.933333,0.367619,1.0,1.0,1031.830021,214.017641
106150,0.666717,0.504397,0.0,0.240000,0.946667,0.368095,1.0,1.0,486.959394,93.668717
106151,0.669875,0.666717,0.0,0.240000,0.960000,0.368571,1.0,1.0,9.473423,103.763368
106152,0.668122,0.669875,0.0,0.240000,0.973333,0.369048,1.0,1.0,-5.259560,93.944570
106153,0.666994,0.668122,0.0,0.240000,0.986667,0.369524,1.0,1.0,-3.383437,82.907071
106154,0.666739,0.666994,0.0,0.240000,1.000000,0.370000,1.0,1.0,-0.764007,81.797599
106155,0.588629,0.666739,0.0,0.241667,0.993333,0.370476,0.8,1.0,-234.331637,40.380647
106156,0.562292,0.588629,0.0,0.243333,0.986667,0.370952,0.8,1.0,-79.010682,46.249515
106157,0.538621,0.562292,0.0,0.245000,0.980000,0.371429,0.8,1.0,-71.012010,53.517810
106158,0.533427,0.538621,0.0,0.246667,0.973333,0.371905,0.8,1.0,-15.583162,66.643734


In [47]:
def get_thermal_nn_model(lr,state_size,pred_size):          
    model = Sequential()
    model.add(Dense(1000,input_dim=state_size,activation="tanh",kernel_initializer = 'he_uniform'))                            
    model.add(Dense(1500, activation="tanh", kernel_initializer='he_uniform'))          
    model.add(Dense(1500, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(1500, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(1500, activation="linear", kernel_initializer='he_uniform'))
    model.add(Dense(pred_size, activation="linear", kernel_initializer='he_uniform'))
    model.summary()
    # loss = tf.keras.losses.Huber(delta=1)
    loss=tf.keras.losses.MeanAbsoluteError()

    # model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07))

    model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07))

    return model  

In [49]:
epochs = 20
batch_size = 1000
path = '0_Data/Case_1/' 
path_NN = path + '02_NN/'

lr = 0.00002

t_model = get_thermal_nn_model(lr=lr,state_size=len(t_model_state_keys),pred_size=2)
t_model.load_weights(path_NN+"t_model.h5")

thermal_history = t_model.fit(therm_X ,therm_y , epochs=epochs, batch_size= batch_size,verbose=1)

t_model.save_weights(path_NN+"t_model.h5")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 1000)              9000      
                                                                 
 dense_43 (Dense)            (None, 1500)              1501500   
                                                                 
 dense_44 (Dense)            (None, 1500)              2251500   


                                                                 
 dense_45 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_46 (Dense)            (None, 1500)              2251500   
                                                                 
 dense_47 (Dense)            (None, 2)                 3002      
                                                                 
Total params: 8,268,002
Trainable params: 8,268,002
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
2734/2734 [==============================] - 20s 7ms/step - loss: 0.8396
Epoch 2/20
2734/2734 [==============================] - 42s 16ms/step - loss: 0.8301
Epoch 3/20
2734/2734 [==============================] - 44s 16ms/step - loss: 0.8269
Epoch 4/20
2734/2734 [==============================] - 44s 16ms/step - loss: 0.8249
Epoch 5/20
2734/2734 [==============================] - 4

In [58]:
lr = 0.000008
path = '0_Data/Case_1/' 
path_NN = path + '02_NN/'


t_model = get_thermal_nn_model(lr=lr,state_size=len(t_model_state_keys),pred_size=2)
t_model.load_weights(path_NN+"t_model.h5")

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 1000)              9000      
_________________________________________________________________
dense_49 (Dense)             (None, 1500)              1501500   
_________________________________________________________________
dense_50 (Dense)             (None, 1500)              2251500   
_________________________________________________________________
dense_51 (Dense)             (None, 1500)              2251500   
_________________________________________________________________
dense_52 (Dense)             (None, 1500)              2251500   
_________________________________________________________________
dense_53 (Dense)             (None, 2)                 3002      
Total params: 8,268,002
Trainable params: 8,268,002
Non-trainable params: 0
____________________________________________

In [19]:
# check prediction 
start = 289809
no = 4 

X_test = therm_X[start:start+no]
t_model.predict(X_test,verbose=0)

array([[ -91.493675,   25.926497],
       [1320.7808  ,   25.680214],
       [ 302.0011  ,   57.591946],
       [  14.174659,   23.52386 ]], dtype=float32)

In [20]:
therm_y[start:start+no]

,delta_T,hvac_dem_tot
289809,-132.400846,26.144518
289810,1277.513551,25.587631
289811,307.838747,55.757171
289812,24.273490,19.642807


In [60]:
start = 300
therm_X[start:start+1]

,Temp_mean_p0,Temp_mean_m1,Occ,temp_OA,rh_OA,hir_sol,u_heat_stp,u_cool_stp
300,0.40396,0.339404,0.0,0.413333,0.76,0.440476,0.8,1.0


In [64]:
X_test = np.array([[0.21134547,0.0302829,  0., 0.32333333,0.85,0.40142857,0.78445607,1]])
t_model.predict(X_test,verbose=0)

array([[740.68506,  75.75097]], dtype=float32)